## Autism Spectrum Disorder 

### Data Scraping

In [11]:
from twitter_scraper_selenium import scrap_keyword

In [21]:
%%time
data = scrap_keyword(keyword="autism", browser="chrome",tweets_count=10000,output_format="csv")

[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST driver version for 100.0.4896
[WDM] - Driver [C:\Users\regmi\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


KeyboardInterrupt: 

In [22]:
import csv

# open the file in the write mode
with open('data.csv', 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(data)

NameError: name 'csv' is not defined